### Optimizing Postgres Databases: Debugging Postgres Queries
#### These are exercises done as part of <a href = "www.dataquest.io"> DataQuest</a>'s Data Engineer Path
This is not replicated for commercial use; strictly personal development.<br>
All exercises are (c) DataQuest, with slight modifications so they use my PostGres server on my localhost

>We will be building on our understanding of the tables and we will learn how to debug Postgres queries. Using internal tables and Postgres sepecific commands, we will analyze how the Postgres engine runs analysis queries. Then, we will use that knowledge to determine what steps we need to take to make them more efficient and faster to run.
>
DataQuest

#### Debugging Postgres Queries
<b>1.  </b>Instructions:
- Use the provided `cur` object.
- Run the `EXPLAIN` command for a `SELECT` all query on the `vbstatic` table.
- Call `.fetchall()` and pretty print the output.

In [2]:
import psycopg2
import pprint as pp

conn = psycopg2.connect(dbname="valenbisi2018", user="nmolivo")
cur = conn.cursor()

cur.execute("EXPLAIN SELECT * FROM vbstatic")
pp.pprint(cur.fetchall())

[('Seq Scan on vbstatic  (cost=0.00..7212.73 rows=273273 width=100)',)]


> Let's describe the path a query takes when you call `cur.execute()`.
>
>Path of a query:
>
>1. The query is parsed for correct syntax. If there are any errors, the query does not execute and you receive an error message. If error-free, then the query is transformed into a query tree.
>
>2. A rewrite system takes the query tree and checks against the system catalog internal tables for any special rules. Then, if there are any rules, it rewrites them into the query tree.
>
>3. The rewritten query tree is then processed by the planner/optimizer which creates a query plan to send to the executor. The planner ensures that this is the fastest possible route for query execution.

>4. The executor takes in the query plan, runs each step, then returns back any rows it found.
>
><img src = "https://github.com/nmolivo/dataquest_eng/blob/master/images/007_sqlpath.png?raw=true"></img><br>
>
>When we run the EXPLAIN command, we are examining the query at the third step in its path. In this step, the planner (or optimizer) is responsible for taking the written query and finding the fastest and most efficient way of returning the results.
>
>DataQuest

<b>2.  </b>Instructions:
- Use the provided `cur` object.
- Run the `EXPLAIN` command on a query that returns a `COUNT` of rows greater than the year 2012-01-01 for `homeless_by_coc`.
- Call `.fetchall()` and <a href="https://docs.python.org/3/library/pprint.html">pretty print</a> the output.

In [5]:
conn = psycopg2.connect(dbname="valenbisi2018", user="nmolivo")
cur = conn.cursor()

cur.execute("EXPLAIN SELECT COUNT(*) from vbstatic WHERE update > '2018-02-22'")
pp.pprint(cur.fetchall())

[('Finalize Aggregate  (cost=7139.46..7139.47 rows=1 width=8)',),
 ('  ->  Gather  (cost=7139.25..7139.46 rows=2 width=8)',),
 ('        Workers Planned: 2',),
 ('        ->  Partial Aggregate  (cost=6139.25..6139.26 rows=1 width=8)',),
 ('              ->  Parallel Seq Scan on vbstatic  (cost=0.00..5903.30 '
  'rows=94380 width=0)',),
 ("                    Filter: (update > '2018-02-22 00:00:00'::timestamp "
  'without time zone)',)]


> The executor will start by running a sequential scan on the table, filter by the year value, and then run the aggregator on those returned results. The plan of execution closely resembles a tree of commands – starting from the bottom and working its way up – but it is not clearly shown by this output format.
>
>DataQuest

<b>3.  </b>Instructions:
- Use the provided `cur` object.
- Run the `EXPLAIN` command on a query that returns a `COUNT` of rows greater than the year 2012-01-01 for `homeless_by_coc`.
    - Format the ouptut with the `json` type.
- Call `.fetchall()` and pretty print the output.

In [6]:
conn = psycopg2.connect(dbname="valenbisi2018", user="nmolivo")
cur = conn.cursor()

cur.execute("EXPLAIN (format json) SELECT COUNT(*) from vbstatic WHERE update > '2018-02-22'")
pp.pprint(cur.fetchall())

[([{'Plan': {'Node Type': 'Aggregate',
             'Parallel Aware': False,
             'Partial Mode': 'Finalize',
             'Plan Rows': 1,
             'Plan Width': 8,
             'Plans': [{'Node Type': 'Gather',
                        'Parallel Aware': False,
                        'Parent Relationship': 'Outer',
                        'Plan Rows': 2,
                        'Plan Width': 8,
                        'Plans': [{'Node Type': 'Aggregate',
                                   'Parallel Aware': False,
                                   'Parent Relationship': 'Outer',
                                   'Partial Mode': 'Partial',
                                   'Plan Rows': 1,
                                   'Plan Width': 8,
                                   'Plans': [{'Alias': 'vbstatic',
                                              'Filter': "(update > '2018-02-22 "
                                                        "00:00:00'::timestamp "
         

<font color = 'blue'><b>Q: What does 'cost' mean?</b><br>
<b>A: </b>The cost of the operation as estimated by the optimizer's cost-based approach. For statements that use the rule-based approach, this column is null. Cost is not determined for table access operations. The value of this column does not have any particular unit of measurement, it is merely a weighted value used to compare costs of execution plans.  
https://docs.oracle.com/cd/A58617_01/server.804/a58246/explan.htm

> The `Startup Cost` represents the time it takes before a rows can be returned (something like sorting, or collecting the rows and aggregating them). <br>
>`Total Cost` includes `Startup Cost` and is the total time it takes to run the node plan until completion. 
>
>DataQuest

<b>4.  </b>Instructions:
- Use the provided `cur` object.
- Practice running different `EXPLAIN` commands on any of the tables in the database. Here is one to try:
    - `SELECT count FROM homeless_by_coc`
    - `SELECT postal FROM state_info WHERE state='Alabama'`
    - `SELECT state, SUM(count) FROM homeless_by_coc GROUP BY state HAVING SUM(count) > 100000 ORDER BY state`

<font color = 'blue'>Here I will query `vbstatic` instead of `homeless_by_coc`</font>

In [12]:
conn = psycopg2.connect(dbname="valenbisi2018", user="nmolivo")
cur = conn.cursor()

cur.execute("EXPLAIN SELECT count(*) FROM vbstatic")
pp.pprint(cur.fetchall())

[('Finalize Aggregate  (cost=6903.51..6903.52 rows=1 width=8)',),
 ('  ->  Gather  (cost=6903.30..6903.51 rows=2 width=8)',),
 ('        Workers Planned: 2',),
 ('        ->  Partial Aggregate  (cost=5903.30..5903.31 rows=1 width=8)',),
 ('              ->  Parallel Seq Scan on vbstatic  (cost=0.00..5618.64 '
  'rows=113864 width=0)',)]


In [14]:
conn = psycopg2.connect(dbname="valenbisi2018", user="nmolivo")
cur = conn.cursor()

cur.execute("EXPLAIN SELECT available FROM vbstatic WHERE free=0")
pp.pprint(cur.fetchall())

[('Seq Scan on vbstatic  (cost=0.00..7895.91 rows=20696 width=4)',),
 ('  Filter: (free = 0)',)]


In [25]:
conn = psycopg2.connect(dbname="valenbisi2018", user="nmolivo")
cur = conn.cursor()

cur.execute("EXPLAIN SELECT available, SUM(free) FROM vbstatic GROUP BY available")
pp.pprint(cur.fetchall())

[('Finalize GroupAggregate  (cost=7199.17..7200.20 rows=41 width=12)',),
 ('  Group Key: available',),
 ('  ->  Sort  (cost=7199.17..7199.38 rows=82 width=12)',),
 ('        Sort Key: available',),
 ('        ->  Gather  (cost=7187.96..7196.57 rows=82 width=12)',),
 ('              Workers Planned: 2',),
 ('              ->  Partial HashAggregate  (cost=6187.96..6188.37 rows=41 '
  'width=12)',),
 ('                    Group Key: available',),
 ('                    ->  Parallel Seq Scan on vbstatic  (cost=0.00..5618.64 '
  'rows=113864 width=8)',)]


> Under the hood, `EXPLAIN` runs several queries on internal tables to give us the estimated data. One of these tables is the `pg_class` table where the estimated costs and rows are stored. This table only stores estimates of rows and costs (not actual values) so `EXPLAIN` can only give us approximate values for our queries.
>
>If we want to see, and force, actual runtime statistics of our queries, we need to use the `ANALYZE` option of the `EXPLAIN` query. With `ANALYZE`, the `EXPLAIN` command will execute our given query, wait for the results, then return the output with the recorded values.
>
>DataQuest & https://www.postgresql.org/docs/9.3/static/sql-explain.html

<b>5.  </b>Instructions:
- Use the provided `cur` object.
- Run the `EXPLAIN` command on a query that returns a `COUNT` of rows greater than the year 2012-01-01 for `homeless_by_coc`.
    - Add the `ANALYZE` option to the `EXPLAIN` command.
    - Format the ouptut with the `json` type.
    - Note that we are trying to add two options for `EXPLAIN`.
- Call `.fetchall()` and pretty print the output.

In [26]:
conn = psycopg2.connect(dbname="valenbisi2018", user="nmolivo")
cur = conn.cursor()

cur.execute("EXPLAIN (ANALYZE, format json) SELECT COUNT(*) from vbstatic WHERE update > '2018-02-22'")
pp.pprint(cur.fetchall())

[([{'Execution Time': 72.429,
    'Plan': {'Actual Loops': 1,
             'Actual Rows': 1,
             'Actual Startup Time': 71.276,
             'Actual Total Time': 71.276,
             'Node Type': 'Aggregate',
             'Parallel Aware': False,
             'Partial Mode': 'Finalize',
             'Plan Rows': 1,
             'Plan Width': 8,
             'Plans': [{'Actual Loops': 1,
                        'Actual Rows': 3,
                        'Actual Startup Time': 71.261,
                        'Actual Total Time': 71.266,
                        'Node Type': 'Gather',
                        'Parallel Aware': False,
                        'Parent Relationship': 'Outer',
                        'Plan Rows': 2,
                        'Plan Width': 8,
                        'Plans': [{'Actual Loops': 3,
                                   'Actual Rows': 1,
                                   'Actual Startup Time': 65.144,
                                   'Actual To

>Using the `ANALYZE` option, we have both estimates and actual times (in milliseconds) side by side. Furthermore, we are presented with the total execution time.

<b>6.  </b>Instructions:
- Use the provided `cur` and conn objects.
- Run the `EXPLAIN ANALYZE` command on a `DELETE` query that deletes all the rows in `state_household_incomes`.
    - Format the ouptut with the `json` type.
- Rollback the delete command.
- Call `.fetchall()` and pretty print the output.

In [28]:
conn = psycopg2.connect(dbname="valenbisi2018", user="nmolivo")
cur = conn.cursor()

cur.execute("EXPLAIN (ANALYZE, FORMAT json) DELETE FROM vbstatic")
# Rollback the change.
conn.rollback()
pp.pprint(cur.fetchall())

[([{'Execution Time': 880.858,
    'Plan': {'Actual Loops': 1,
             'Actual Rows': 0,
             'Actual Startup Time': 880.032,
             'Actual Total Time': 880.032,
             'Alias': 'vbstatic',
             'Node Type': 'ModifyTable',
             'Operation': 'Delete',
             'Parallel Aware': False,
             'Plan Rows': 273273,
             'Plan Width': 6,
             'Plans': [{'Actual Loops': 1,
                        'Actual Rows': 273273,
                        'Actual Startup Time': 0.097,
                        'Actual Total Time': 101.738,
                        'Alias': 'vbstatic',
                        'Node Type': 'Seq Scan',
                        'Parallel Aware': False,
                        'Parent Relationship': 'Member',
                        'Plan Rows': 273273,
                        'Plan Width': 6,
                        'Relation Name': 'vbstatic',
                        'Startup Cost': 0.0,
                       

<b>7.  </b>Instructions:
- Use the provided `cur` object.
- Run `EXPLAIN ANALYZE` on a select from `homeless_by_coc` and `state_info`:
    - Select columns `state`, `coc_number`, and `coc_name` from `homeless_by_coc`.
    - Select column name from `state_info`.
    - Join on `homless_by_coc.state` and `state_info.postal`.
    - Format the ouptut with the `json` type.
- Call `fetchall()` and pretty print the output.

```python
conn = psycopg2.connect(dbname="dq", user="hud_admin", password="abc123")
cur = conn.cursor()

cur.execute("EXPLAIN (ANALYZE, FORMAT json) SELECT hbc.state, hbc.coc_number, hbc.coc_name, si.name FROM homeless_by_coc as hbc, state_info as si WHERE hbc.state = si.postal")
pp.pprint(cur.fetchall())
```

><img src = 'https://github.com/nmolivo/dataquest_eng/blob/master/images/008_query_plan.png?raw=true'></img><br>
> The output of the `EXPLAIN ANALYZE` command reveals the inefficiency of the join. In the list of plans, each node must first run a Seq Scan which is a loop through each of the tables. Before the join can occur, a loop is performed twice: once in `homeless_by_coc` and once in `state_info`.
>
>DataQuest